exact solver求解，需要由SBML模型生成求解器可读的模型输入文件，分别写入目标函数，约束和变量（反应边界），所有反应名称前加字符r以免出现数字开头的反应名称造成无法计算的错误，浮点数的系数均转换成有理数如20568/9223372进行精确求解。通过该求解器可以解决浮点数计算误差的问题，准确得出最优途径中的反应个数。

In [33]:
from fractions import Fraction
import subprocess
#from __future__ import print_function
#import pandas as pd
import cobra
import cobra.test #内嵌模型
mname="ecoli"
#mname="textbook"
model = cobra.test.create_test_model(mname)
outputfile=mname+'_flux.txt' #输出通量
lpfilename=mname+'.lp' #exact solver需要的模型输入文件格式

所有反应名称前加字符r以免出现数字开头的反应名称造成无法计算的错误，

In [34]:
for rea in model.reactions:
    rea.id='r'+rea.id

生成esolver需要的输入文件，分别写入目标函数，约束和变量（反应边界）

In [35]:
obj=str(model.objective.expression).replace('*',' ')
print(obj)
#obj: -1.0 Ec_biomass_iJO1366_reverse_e94eb + 1.0 Ec_biomass_iJO1366 cobrapy中每个反应都拆成两个方向相反反应
optobj = 'Maximize\nobj: '+obj+ '\n' 

-1.0 rEc_biomass_iJO1366_core_53p95M_reverse_a239e + 1.0 rEc_biomass_iJO1366_core_53p95M


处理针对每个代谢物的计量约束，如0 <= 1.0*GAPD - 1.0*GAPD_reverse_459c1 + 1.0*PGK - 1.0*PGK_reverse_02696 <= 0
转换为1 GAPD -1 GAPD_reverse_459c1 + 1 PGK -1 PGK_reverse_02696 = 0的exact solver模型约束条件格式，对每个系数转换为有理数.
对非等式约束条件必须拆分写成 1 x1 - 2 x2 >=0,1 x1 - 2 x2 <=0的形式，不能直接写0 <= 1 x1 - 2 x2 <=0,也不能变量在后如0 <= 1 x1 - 2 x2

In [37]:
cons=model.constraints
constraint = 'Subject To\n'
for con in cons:
    a=con.get_linear_coefficients(con.variables)
    rea=[]
    for k,v in a.items():
        rea.append(str(Fraction(v))+' '+k.name)#所有反应名称前加r以免出现数字开头的反应名称造成错误
    connew=' + '.join(rea)
    constraint +=  connew + ' = 0\n' #对代谢分析可写成等式形式
#    constraint +=  connew + ' >= '+str(con.lb)+'\n'
#    constraint += connew + ' <= '+str(con.ub)+'\n'
constraint =constraint.replace('+ -', '-') #将 + -1 的情况改为-1    rea=[]

下面是一种直接针对字符串进行处理的方法，未对反应名称进行更改，如果反应名称首字符为数字可能出错，不建议用

In [ ]:
cons=model.constraints
constraint = 'Subject To\n'
for con in cons:
    rea=[]
    reas=str(con.expression).split(" + ")
    for reas2 in reas:
        reas3=reas2.split(" - ")
        rcoef=reas3[0].split("*") #split coefficients
        rea.append(str(Fraction(float(rcoef[0])))+' '+rcoef[1])
        for r in reas3[1:]:
            rcoef=r.split("*")
            rea.append(str(Fraction(-float(rcoef[0])))+' '+rcoef[1])
    connew=' + '.join(rea)
    constraint += connew + ' = 0\n'
#    constraint += '0 <= '+ connew + ' <= 0\n'
constraint =constraint.replace('+ -', '-') #将 + -1 的情况改为-1

In [38]:
bounds = 'Bounds\n'
rbounds=model.variables
bounds += ''.join(str(bound) + '\n' for bound in rbounds)
f = open(lpfilename, 'w') #输出lp文件
f.write(optobj+constraint+bounds+'End\n')
f.close()

In [39]:
subprocess.call(["esolver", "-Ooptimal", lpfilename]) #调用esolver进行计算
subprocess.call(["gunzip", "optimal.sol.gz"])  #输出结果解压
outFile = 'optimal.sol'  #myadd

下面对esolver的输出文件进行处理，输出最优途径中各反应的通量值

In [40]:
f = open(outFile, 'r')
g = f.readlines()
f.close()
flux = open(outputfile, 'w')
opt1=g[2].split('=')[1].strip()
if '/' in opt1:
	nums=opt1.split('/')
	optrate=float(nums[0])/float(nums[1])
else:
	optrate=float(opt1)
flux.write('optimal rate:\t'+str(optrate)+'\n')
for k in range(4,len(g)):
    if g[k]== 'REDUCED COST:\n':
        break
    else:
        rflux=g[k].split('=')
        rname=rflux[0].strip()
        opt1=rflux[1].strip()
        if '/' in opt1:
            nums=opt1.split('/')
            if 'reverse' in rname:
                optrate=-float(nums[0])/float(nums[1])
                rname=rname.split('_')[0]
            else:
                optrate=float(nums[0])/float(nums[1])
        else:
            optrate=float(opt1)
            if 'reverse' in rname:
                rname=rname.split('_')[0]
        flux.write(rname+'\t'+str(optrate)+'\n')
#        flux.write(g[k].strip('\n')+'\t'+str(optrate)+'\n')
flux.close()
subprocess.call(["rm", outFile])


0